In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.io import loadmat
from spectral import *
from functions import *
import multiprocessing
import warnings
import time
import math

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
def causal_get_cov_loc(order=[2,2]):
    cov_loc = []
    p = order[0]
    q = order[1]
    for i in range(0,-p-1,-1):
        for j in range(0,i,-1):
            if((i==0) and (j==0)): continue
            cov_loc.append([i,j])
        if(q + i >= 0):
            for j in range(i,1):
                if((i==0) and (j==0)): continue
                cov_loc.append([j,i])
    return cov_loc

### order selection

In [ ]:
order_list = [[2,2], [3,3], [4,4], [5,5], [6,5], [6,6], [7,7]]
max_order = [7,7]
[p_max, q_max] = max_order
nrep = 20    ### Change "nrep = 1" for quick result
SEED = 2024

cov_loc = causal_get_cov_loc(max_order)

MSE_lev_list = np.ones(len(order_list))
MSE_fast_list = np.ones(len(order_list))
MSE_OLS_list = np.ones(len(order_list))
MSE_random_list = np.ones(len(order_list))
MSE_lev_std_list = np.zeros(len(order_list))
MSE_fast_std_list = np.zeros(len(order_list))
MSE_OLS_std_list = np.zeros(len(order_list))
MSE_random_std_list = np.zeros(len(order_list))

MSE_para_lev_list = np.ones(len(order_list))
MSE_para_fast_list = np.ones(len(order_list))
MSE_para_OLS_list = np.ones(len(order_list))
MSE_para_random_list = np.ones(len(order_list))
MSE_para_lev_std_list = np.zeros(len(order_list))
MSE_para_fast_std_list = np.zeros(len(order_list))
MSE_para_OLS_std_list = np.zeros(len(order_list))
MSE_para_random_std_list = np.zeros(len(order_list))

BIC_lev_list = np.zeros([p_max+1, q_max+1, len(order_list)])
BIC_fast_list = np.zeros([p_max+1, q_max+1, len(order_list)])
BIC_OLS_list = np.zeros([p_max+1, q_max+1, len(order_list)])
BIC_random_list = np.zeros([p_max+1, q_max+1, len(order_list)])
AIC_lev_list = np.zeros([p_max+1, q_max+1, len(order_list)])
AIC_fast_list = np.zeros([p_max+1, q_max+1, len(order_list)])
AIC_OLS_list = np.zeros([p_max+1, q_max+1, len(order_list)])
AIC_random_list = np.zeros([p_max+1, q_max+1, len(order_list)])

time_lev_list = np.zeros(len(order_list))
time_fast_list = np.zeros(len(order_list))
time_OLS_list = np.zeros(len(order_list))
time_random_list = np.zeros(len(order_list))

m = 300
n = 100
subsample_size = 1000

order_num = 0
para_num_list = []

for order in order_list:
    p = order[0]
    q = order[1]
    beta = beta_generate(order, method = '3')
    phi = beta_to_phi(beta, order)
    para_num = (p+1)*(q+1)-1
    para_num_list.append(para_num)
    print(order)

    MSE_para_lev = np.zeros(nrep)
    MSE_para_fast = np.zeros(nrep)
    MSE_para_OLS = np.zeros(nrep)
    MSE_para_random = np.zeros(nrep)

    for rep in range(nrep):

        np.random.seed(rep)
        Y = np.zeros([m,n])
        for i in range(p_max):
            Y[i,:] = 1*np.random.randn(n) + i
        for i in range(q_max):
            Y[:,i] = 1*np.random.randn(m) + i

        for i in range(p_max,m):
            for j in range(q_max,n):
                Y[i,j] = np.sum(Y[(i-p):(i+1),(j-q):(j+1)] * beta) + 0.1 * np.sum(np.random.randn(1))

        cov_loc = causal_get_cov_loc(max_order)
        MSE_lev_temp = np.ones(len(cov_loc))
        MSE_fast_temp = np.ones(len(cov_loc))
        MSE_OLS_temp = np.ones(len(cov_loc))
        MSE_random_temp = np.ones(len(cov_loc))

        MSE0 = np.mean((Y[p_max:,q_max:] - np.mean(Y[p_max:,q_max:]))**2 )
        XX = np.zeros([(m-p_max), (n-q_max),len(cov_loc)])
        i = 0
        for loc in cov_loc:
            k = loc[0]
            l = loc[1]
            XX[:,:,i] = Y[(p_max+k):(m+k), (q_max+l):(n+l)]
            i = i + 1

        start_lev = time.time()
        BIC_lev = np.zeros([p_max+1, q_max+1])
        AIC_lev = np.zeros([p_max+1, q_max+1])
        AIC_lev[0,0] = AIC((m-p_max)*(n-q_max), 0, MSE0)
        BIC_lev[0,0] = BIC((m-p_max)*(n-q_max), 0, MSE0)
        AIC_temp = np.zeros(len(cov_loc))
        AIC_min = 10
        for l in range(len(cov_loc)):
            loc = cov_loc[l]
            XX_temp = XX[:,:,:(l+1)]
            lev_temp = np.zeros([m-p_max,n-q_max])
            matrix_inv = np.linalg.inv(tensor_multiply(XX_temp, XX_temp))
            for i in range(m-p_max):
                for j in range(n-q_max):
                    lev_temp[i,j] = XX_temp[i,j,:].T @ matrix_inv @ XX_temp[i,j,:]
            phi_lev = two_dim_param_estimate_leverage(XX_temp, Y[p_max:,q_max:], lev_score = lev_temp, subsample_size = subsample_size)
            MSE_lev_temp[l] = MSE_calculate(XX_temp, Y[p_max:,q_max:], phi_lev)
            BIC_lev[-loc[0], -loc[1]] = BIC((m-p_max)*(n-q_max), l+1, MSE_lev_temp[l])
            AIC_lev[-loc[0], -loc[1]] = AIC((m-p_max)*(n-q_max), l+1, MSE_lev_temp[l])
            AIC_temp[l] = AIC((m-p_max)*(n-q_max), l+1, MSE_lev_temp[l])
            if(AIC_temp[l] < AIC_min):
                AIC_min = AIC_temp[l]
                phi_aic = phi_lev
        end_lev = time.time()
        time_lev = end_lev - start_lev
        time_lev_list[order_num] = time_lev
        index = np.where(AIC_temp == np.min(AIC_temp))[0][0]
        MSE_lev_list[order_num] = MSE_lev_temp[index]
        phi_expand = np.zeros(len(cov_loc))
        phi_aic_expand = np.zeros(len(cov_loc))
        phi_expand[:len(phi)] = phi
        phi_aic_expand[:len(phi_aic)] = phi_aic
        MSE_para_lev[rep] = np.mean((phi_expand - phi_aic_expand)**2)

        start_fast = time.time()
        BIC_fast = np.zeros([p_max+1, q_max+1])
        AIC_fast = np.zeros([p_max+1, q_max+1])
        AIC_fast[0,0] = AIC((m-p_max)*(n-q_max), 0, MSE0)
        BIC_fast[0,0] = BIC((m-p_max)*(n-q_max), 0, MSE0)
        AIC_temp = np.zeros(len(cov_loc))
        AIC_min = 10
        leverage_score_list = causal_fast_order_selection(Y, order=max_order, subsample_size = subsample_size)
        for l in range(len(cov_loc)):
            loc = cov_loc[l]
            XX_temp = XX[:,:,:(l+1)]
            lev_temp = leverage_score_list[l]
            phi_fast = two_dim_param_estimate_leverage(XX_temp, Y[p_max:,q_max:], lev_score = leverage_score_list[l], subsample_size = subsample_size)
            MSE_fast_temp[l] = MSE_calculate(XX_temp, Y[p_max:,q_max:], phi_fast)
            BIC_fast[-loc[0], -loc[1]] = BIC((m-p_max)*(n-q_max), l+1, MSE_fast_temp[l])
            AIC_fast[-loc[0], -loc[1]] = AIC((m-p_max)*(n-q_max), l+1, MSE_fast_temp[l])
            AIC_temp[l] = AIC((m-p_max)*(n-q_max), l+1, MSE_fast_temp[l])
            if(AIC_temp[l] < AIC_min):
                AIC_min = AIC_temp[l]
                phi_aic = phi_fast
        end_fast = time.time()
        time_fast = end_fast - start_fast
        time_fast_list[order_num] = time_fast
        index = np.where(AIC_temp == np.min(AIC_temp))[0][0]
        MSE_fast_list[order_num] = MSE_fast_temp[index]
        phi_expand = np.zeros(len(cov_loc))
        phi_aic_expand = np.zeros(len(cov_loc))
        phi_expand[:len(phi)] = phi
        phi_aic_expand[:len(phi_aic)] = phi_aic
        MSE_para_fast[rep] = np.mean((phi_expand - phi_aic_expand)**2)

        start_OLS = time.time()
        BIC_OLS = np.zeros([p_max+1, q_max+1])
        AIC_OLS = np.zeros([p_max+1, q_max+1])
        AIC_OLS[0,0] = AIC((m-p_max)*(n-q_max), 0, MSE0)
        BIC_OLS[0,0] = BIC((m-p_max)*(n-q_max), 0, MSE0)
        AIC_temp = np.zeros(len(cov_loc))
        AIC_min = 10
        for l in range(len(cov_loc)):
            loc = cov_loc[l]
            XX_temp = XX[:,:,:(l+1)]
            phi_OLS = two_dim_param_estimate_leverage(XX_temp, Y[p_max:,q_max:]).reshape(-1)
            MSE_OLS_temp[l] = MSE_calculate(XX_temp, Y[p_max:,q_max:], phi_OLS)
            BIC_OLS[-loc[0], -loc[1]] = BIC((m-p_max)*(n-q_max), l+1, MSE_OLS_temp[l])
            AIC_OLS[-loc[0], -loc[1]] = AIC((m-p_max)*(n-q_max), l+1, MSE_OLS_temp[l])
            AIC_temp[l] = AIC((m-p_max)*(n-q_max), l+1, MSE_OLS_temp[l])
            if(AIC_temp[l] < AIC_min):
                AIC_min = AIC_temp[l]
                phi_aic = phi_OLS
        end_OLS = time.time()
        time_OLS = end_OLS - start_OLS
        time_OLS_list[order_num] = time_OLS
        index = np.where(AIC_temp == np.min(AIC_temp))[0][0]
        MSE_OLS_list[order_num] = MSE_OLS_temp[index]
        phi_expand = np.zeros(len(cov_loc))
        phi_aic_expand = np.zeros(len(cov_loc))
        phi_expand[:len(phi)] = phi
        phi_aic_expand[:len(phi_aic)] = phi_aic
        MSE_para_OLS[rep] = np.mean((phi_expand - phi_aic_expand)**2)

        start_random = time.time()
        MSE_random_temp = np.ones(len(cov_loc))
        BIC_random = np.zeros([p_max+1, q_max+1])
        AIC_random = np.zeros([p_max+1, q_max+1])
        AIC_temp = np.zeros(len(cov_loc))
        AIC_random[0,0] = AIC((m-p_max)*(n-q_max), 0, MSE0)
        BIC_random[0,0] = BIC((m-p_max)*(n-q_max), 0, MSE0)
        AIC_min = 10
        for l in range(len(cov_loc)):
            loc = cov_loc[l]
            XX_temp = XX[:,:,:(l+1)]
            phi_random = two_dim_param_estimate_leverage(XX_temp, Y[p_max:,q_max:], lev_score = np.ones([m-p_max,n-q_max]), subsample_size = subsample_size)
            MSE_random_temp[l] = MSE_calculate(XX_temp, Y[p_max:,q_max:], phi_random)
            BIC_random[-loc[0], -loc[1]] = BIC((m-p_max)*(n-q_max), l+1, MSE_random_temp[l])
            AIC_random[-loc[0], -loc[1]] = AIC((m-p_max)*(n-q_max), l+1, MSE_random_temp[l])
            AIC_temp[l] = AIC((m-p_max)*(n-q_max), l+1, MSE_random_temp[l])
            if(AIC_temp[l] < AIC_min):
                AIC_min = AIC_temp[l]
                phi_aic = phi_random
        end_random = time.time()
        time_random = end_random - start_random
        time_random_list[order_num] = time_random
        index = np.where(AIC_temp == np.min(AIC_temp))[0][0]
        MSE_random_list[order_num] = MSE_random_temp[index]
        phi_expand = np.zeros(len(cov_loc))
        phi_aic_expand = np.zeros(len(cov_loc))
        phi_expand[:len(phi)] = phi
        phi_aic_expand[:len(phi_aic)] = phi_aic
        MSE_para_random[rep] = np.mean((phi_expand - phi_aic_expand)**2)

    MSE_para_lev_list[order_num] = np.mean(MSE_para_lev)
    MSE_para_fast_list[order_num] = np.mean(MSE_para_fast)
    MSE_para_OLS_list[order_num] = np.mean(MSE_para_OLS)
    MSE_para_random_list[order_num] = np.mean(MSE_para_random)

    MSE_para_lev_std_list[order_num] = np.std(MSE_para_lev)
    MSE_para_fast_std_list[order_num] = np.std(MSE_para_fast)
    MSE_para_OLS_std_list[order_num] = np.std(MSE_para_OLS)
    MSE_para_random_std_list[order_num] = np.std(MSE_para_random)

    AIC_lev_list[:,:,order_num] = AIC_lev
    AIC_fast_list[:,:,order_num] = AIC_fast
    AIC_OLS_list[:,:,order_num] = AIC_OLS
    AIC_random_list[:,:,order_num] = AIC_random

    order_num = order_num + 1

In [ ]:
plt.errorbar(para_num_list, time_lev_list, color = 'orange', marker = 'o', label = 'LEV-exact')
plt.errorbar(para_num_list, time_fast_list, color = 'red', marker = '^', label = 'LEV-appr')
plt.errorbar(para_num_list, time_OLS_list, color = 'black', marker = 's', label = 'OLS')
plt.errorbar(para_num_list, time_random_list, color = 'blue', marker = '.', label = 'UNIF')
#plt.xscale('log')
plt.yscale('log')
plt.xlabel('number of parameter d', fontsize = 20)
plt.ylabel('time cost (sec)', fontsize = 20)
plt.tick_params(axis='both', labelsize=14)
plt.legend()

In [ ]:
t = 3
print(order_list[t])
print(np.where(AIC_OLS_list[:,:,t] == np.min(AIC_OLS_list[:,:,t])))
plt.imshow(AIC_OLS_list[:,:,t].T)
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
print(order_list[t])
print(np.where(AIC_fast_list[:,:,t] == np.min(AIC_fast_list[:,:,t])))
plt.imshow(AIC_fast_list[:,:,t].T)
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
print(order_list[t])
print(np.where(AIC_lev_list[:,:,t] == np.min(AIC_lev_list[:,:,t])))
plt.imshow(AIC_lev_list[:,:,t].T)
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
print(order_list[t])
print(np.where(AIC_random_list[:,:,t] == np.min(AIC_random_list[:,:,t])))
plt.imshow(AIC_random_list[:,:,t].T)
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.errorbar(para_num_list, MSE_para_lev_list, yerr = MSE_para_lev_std_list, color = 'orange', fmt = 'o-', label = 'LEV-exact')
plt.errorbar(para_num_list, MSE_para_fast_list, yerr = MSE_para_fast_std_list, color = 'red', fmt = '^-', label = 'LEV-appr')
plt.errorbar(para_num_list, MSE_para_OLS_list, yerr = MSE_para_OLS_std_list, color = 'black', fmt = 's-', label = 'OLS')
plt.errorbar(para_num_list, MSE_para_random_list, yerr = MSE_para_random_std_list, color = 'blue', fmt = '.-', label = 'UNIF')
plt.yscale('log')
plt.xlabel('number of parameter d', fontsize = 20)
plt.ylabel('MSE', fontsize = 20)
plt.ylim([1e-6,1e-4])
plt.tick_params(axis='both', labelsize=14)
plt.legend()